In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools
%matplotlib inline

In [13]:
raw_data = pd.read_csv('2014-2015.csv',sep=',')
raw_data.head(10)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,I1,30/08/14,Chievo,Juventus,0,1,A,0,1,A,...,1.80,27,1.00,2.00,1.94,1.98,1.92,8.35,3.84,1.56
1,I1,30/08/14,Roma,Fiorentina,2,0,H,1,0,H,...,1.94,21,-0.75,2.00,1.95,1.98,1.92,1.73,3.80,5.56
2,I1,31/08/14,Atalanta,Verona,0,0,D,0,0,D,...,1.70,21,-0.50,2.15,2.10,1.84,1.80,2.07,3.47,3.99
3,I1,31/08/14,Cesena,Parma,1,0,H,1,0,H,...,1.59,21,0.25,1.91,1.86,2.08,2.01,5.44,3.65,1.76
4,I1,31/08/14,Genoa,Napoli,1,2,A,1,1,D,...,1.82,21,0.50,1.91,1.87,2.04,2.01,3.92,3.54,2.07
5,I1,31/08/14,Milan,Lazio,3,1,H,1,0,H,...,1.79,20,-0.50,2.05,2.00,1.90,1.87,2.13,3.18,4.28
6,I1,31/08/14,Palermo,Sampdoria,1,1,D,1,0,H,...,1.65,21,-0.25,1.96,1.89,2.03,1.98,2.26,3.29,3.64
7,I1,31/08/14,Sassuolo,Cagliari,1,1,D,1,1,D,...,2.10,21,0.00,1.72,1.67,2.36,2.25,2.03,3.71,3.89
8,I1,31/08/14,Torino,Inter,0,0,D,0,0,D,...,1.77,21,0.00,2.38,2.28,1.72,1.66,2.77,3.34,2.78
9,I1,31/08/14,Udinese,Empoli,2,0,H,0,0,D,...,1.74,21,-0.75,2.10,2.05,1.87,1.83,1.77,3.69,5.48


In [14]:
columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']

playing_statistics = raw_data[columns_req]

In [16]:
# Gets the goals scored agg arranged by teams and matchweek
def get_goals_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



# Gets the goals conceded agg arranged by teams and matchweek
def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat


# Apply to each dataset
playing_statistics = get_gss(playing_statistics)

<ipython-input-16-8c4cba2ba8be>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['HTGS'] = HTGS
<ipython-input-16-8c4cba2ba8be>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['ATGS'] = ATGS
<ipython-input-16-8c4cba2ba8be>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html